<a href="https://colab.research.google.com/github/venkat2ram/ML_From_Scratch/blob/master/NeuralNet_From_Scratch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
data=pd.read_csv('/data_banknote_authentication.csv')

In [6]:
data.head()

,a,b,c,d,y
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [0]:
X=data[['a','b','c','d']].to_numpy().T
Y=data[['y']].to_numpy().reshape(1,-1)

In [8]:
X

array([[  3.6216 ,   4.5459 ,   3.866  , ...,  -3.7503 ,  -3.5637 ,
         -2.5419 ],
       [  8.6661 ,   8.1674 ,  -2.6383 , ..., -13.4586 ,  -8.3827 ,
         -0.65804],
       [ -2.8073 ,  -2.4586 ,   1.9242 , ...,  17.5932 ,  12.393  ,
          2.6842 ],
       [ -0.44699,  -1.4621 ,   0.10645, ...,  -2.7771 ,  -1.2823 ,
          1.1952 ]])

In [0]:
def init_params(layer_dims):
  np.random.seed(3)
  L=len(layer_dims)
  params={}
  for l in range(1,L):
    params['w'+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])
    #params['w'+str(l)]=np.ones((layer_dims[l],layer_dims[l-1]))
    params['b'+str(l)]=np.zeros((layer_dims[l],1))
  return params


In [0]:
def sigmoid(Z):
  A=1/(1+np.exp(np.dot(-1,Z)))
  cache=(Z)
  return A,cache

In [0]:
def forward_prop(X,params):
  A=X
  caches=[]
  L=len(params)//2
  for l in range(1,L+1):
    A_prev=A
    Z=np.dot(params['w'+str(l)],A_prev)+params['b'+str(l)]
    linear_cache=(A_prev,params['w'+str(l)],params['b'+str(l)])
    A,activation_cache=sigmoid(Z)
    cache=(linear_cache,activation_cache)
    caches.append(cache)
  return A,caches   


In [0]:
def cost_function(A, Y):
  m=Y.shape[1]
  cost=(-1/m)*np.sum(np.dot(np.log(A),Y.T)+np.dot(np.log(1-A),1-Y.T))
  return cost

In [0]:
def one_layer_backward(dA,cache):
  linear_cache, activation_cache=cache
  
  Z=activation_cache
  sig,c=sigmoid(Z)
  dZ=dA*sig*(np.ones(Z.shape)-sig)

  A_prev,W,b=linear_cache
  m=A_prev.shape[1]
  dW=(1/m)*(np.dot(dZ,A_prev.T))
  dB=(1/m)*(np.sum(dZ,axis=1,keepdims=True))
  dA_prev=(1/m)*np.dot(W.T,dZ)

  return dA_prev,dW,dB

In [0]:
def backprop(AL,Y,caches):
  grads={}
  L=len(caches)
  Y = Y.reshape(AL.shape)
  dA=-1*(np.divide(Y,AL)+np.divide(1-Y,1-AL))
  grads['dA'+str(L-1)],grads['dW'+str(L-1)],grads['dB'+str(L-1)]=one_layer_backward(dA,caches[L-1])
  for l in reversed(range(L-1)):
    #print(l)
    grads['dA'+str(l)],grads['dW'+str(l)],grads['dB'+str(l)]=one_layer_backward(grads['dA'+str(l+1)],caches[l])
  return grads


In [0]:
def update_parameters(parameters, grads, learning_rate):
  L=len(parameters)//2
  for l in range(1,L):
    parameters['w'+str(l)]=parameters['w'+str(l)]-learning_rate*grads['dW'+str(l-1)]
    parameters['b'+str(l)]=parameters['b'+str(l)]-learning_rate*grads['dB'+str(l-1)]
  return parameters


In [0]:
def train(X, Y, layer_dims, epochs, lr):
    params = init_params(layer_dims)
    cost_history = []
    
    for i in range(epochs):
        Y_hat, caches = forward_prop(X, params)
        cost = cost_function(Y_hat, Y)
        cost_history.append(cost)
        grads = backprop(Y_hat, Y, caches)
        
        params = update_parameters(params, grads, lr)
        if(i%15000==1):
          print(cost)
        
        
    return params, cost_history

In [167]:
layer_dims=[4,2,1]
p,c=train(X,Y,layer_dims,150000,1)

0.7410930844359469
0.729122305091288
0.7098744862264953
0.7050869492960836
0.7018639356381856
0.6996342675129796
0.6979580273161388
0.6966213172356486
0.6955160028988366
0.6945809333463774


In [157]:
p

{'b1': array([[-1.89161397],
        [-2.6577572 ],
        [-0.15684041]]),
 'b2': array([[0.]]),
 'w1': array([[-0.04462166, -1.26649387, -1.73238929, -0.51962403],
        [-0.26622896, -0.72292179, -1.15658356,  0.73876534],
        [-0.26309769, -0.66857258, -1.31957786,  0.90468082]]),
 'w2': array([[0.88131804, 1.70957306, 0.05003364]])}

In [0]:
A,c=forward_prop(X,p)

In [0]:
A[A<=0.37]=0
A[A>0.37]=1

In [164]:
sum(sum((A-Y)**2))

762.0

In [81]:
sum(sum(A))

1202.1927881157972

In [160]:
A

array([[0.50081169, 0.50036727, 0.53369761, ..., 0.5000005 , 0.50000931,
        0.51274679]])

In [161]:
Y

array([[0, 0, 0, ..., 1, 1, 1]])

**Rough**

In [19]:
vvvvvvvvvvY.shape**********

SyntaxError: ignored

In [0]:
params=init_params(layer_dim)

In [0]:
y_hat,caches=forward_prop(X,params)

In [0]:
caches

In [0]:
grads=backprop(y_hat,Y,caches)

In [0]:
params

In [0]:
grads['dA2'].reshape(1,-1)

In [0]:
grads['dA2']

In [0]:
caches

In [0]:
dA1=np.array([[ 2.40363348e+08,  2.40363219e+08,  2.61996268e+08],
        [ 2.15179835e+07,  2.15175481e+07,  2.34541597e+07],
        [-3.31390135e+08, -3.31390064e+08, -3.61215665e+08],
        [ 5.11541584e+07,  5.11539908e+07,  5.57579265e+07]])

In [0]:
Z=np.array([[0.94499721, 0.94499721, 0.95864549],
          [0.94499721, 0.94499721, 0.95864549],
          [0.94499721, 0.94499721, 0.95864549],
          [0.94499721, 0.94499721, 0.95864549]])

In [0]:
sig,xs=sigmoid(Z)
dZ=dA*sig*(np.ones(Z.shape)-sig)

In [0]:
dZ

In [0]:
grads['dA4']

In [0]:
len(caches)

In [0]:
dA=-1*(np.divide(Y,y_hat)+np.divide(1-Y,1-y_hat))

In [0]:
dA

In [0]:
np.sum(dA,axis=1,keepdims=True)

In [0]:
grads={}

In [0]:
grads['dA'+str(2)],grads['dW'+str(2)],grads['dB'+str(2)]=one_layer_backward(dA,caches[2])

In [0]:
grads['dB'+str(0)]

In [0]:
np.sum(grads['dB'+str(0)],axis=2)

In [0]:
grads['dA'+str(1)],grads['dW'+str(1)],grads['dB'+str(1)]=one_layer_backward(grads['dA'+str(2)],cache[l])

In [0]:
caches

In [0]:
y_hat

In [0]:
dA=-1*np.divide(b,y_hat)+np.divide(1-b,1-y_hat)

In [0]:
one_layer_backward(dA,cache[2])

In [0]:
params

In [0]:
cache

In [0]:
params

In [0]:
grads=

In [0]:
np.dot(params['w1'],a)

In [0]:
sigmoid(2)

In [0]:
c1,c2=cache[2]

In [0]:
c2

In [0]:
dA

In [0]:
dA*sigmoid(c2)*sigmoid(1-c2)

In [0]:
d1*sigmoid(d2)*(np.ones(d2.shape)-sigmoid(d2))

In [0]:
sigmoid(d2)

In [0]:
np.ones(d2.shape)-sigmoid(d2)

In [0]:
len(caches)

In [0]:
a1=np.array([[[1,2],[3,4]],[[1,2],[3,4]]])

In [0]:
np.sum(a1,axis=0)

In [0]:
ax=np.array([[[1.82966561e+71, 5.37101649e+34, 3.10119951e+31],
         [1.82966561e+71, 5.37101649e+34, 3.10119951e+31],
         [1.82966561e+71, 5.37101649e+34, 3.10119951e+31],
         [1.82966561e+71, 5.37101649e+34, 3.10119951e+31]],
 
        [[3.14106226e+80, 3.07004636e+50, 7.37319456e+50],
         [3.14106226e+80, 3.07004636e+50, 7.37319456e+50],
         [3.14106226e+80, 3.07004636e+50, 7.37319456e+50],
         [3.14106226e+80, 3.07004636e+50, 7.37319456e+50]]])

In [0]:
np.sum(a3,axis=0)

In [0]:
axx=np.array([[[ 1,2,3]],
 
        [[1,2,3]]])

In [0]:
params

In [92]:
np.random.randn(3,3)

array([[ 1.12397796, -0.13191423, -1.62328545],
       [ 0.64667545, -0.35627076, -1.74314104],
       [-0.59664964, -0.58859438, -0.8738823 ]])